In [21]:
import numpy as np
import pandas as pd
from collections import Counter
import re

In [57]:
class BestGuess:
    def __init__ (self):
        self.all_words = list(pd.read_csv('words.csv')['aahed'])
        self.reg_n = ['^[^&', '[^&', '[^&', '[^&', '[^&', '$']
        self.secured = ['.', '.', '.', '.', '.']
        self.guess = ''
        self.filtered_words = self.all_words
        

    @staticmethod
    def generate_pattern(base, x):
        # Produces the feedback string
        counts = Counter(base)
        results = []
        for i, letter in enumerate(x):
            if x[i] == base[i]:
                results += x[i]
                counts[x[i]] -= 1
            else:
                results += '+'
        for i, letter in enumerate(x):
            if x[i] != base[i] and x[i] in base:
                if counts[x[i]] > 0:
                    counts[x[i]] -= 1
                    results[i] = '-'
        return ''.join(results)

    
    @staticmethod
    def entropy(vals, total):
        x = list(vals.values())
        x = np.array(x) / total
        entropy = -sum(x * np.log2(x))
        return entropy

    def highest_entropy_word(self):
        list_words = {w: Counter([self.generate_pattern(f, ''.join(w)) for f in self.filtered_words]) for w in self.filtered_words}
        tot_words = len(list_words)
        entropies = {i: self.entropy(list_words[i], total=tot_words) for i in list_words}
        word = max(entropies, key=entropies.get)
        return word, entropies[word]

    def regex_exp_one(self, res):
        l_s, words_seen = '', []
        for i, n in enumerate(res):
            if n == '+':
                if self.guess[i] not in words_seen:
                    for letter in range(5):
                        if n != self.secured[letter]:
                            self.reg_n[letter] += self.guess[i]
            elif n == '-':

                self.reg_n[i] += self.guess[i]
                l_s += f'(?=.*{self.guess[i]}.*)'
                words_seen.append(self.guess[i])
            elif n == '-' and self.guess[i] in words_seen:
                l_s += f'(?=.*{self.guess[i]}.*{self.guess[i]}.*)'
            else:
                self.secured[i] = self.guess[i]
        re_str = l_s + '^' + ''.join(self.secured) + '$'
        return re.compile(re_str)

    def regex_exp_two(self):
        final = ''
        for i, n in enumerate(self.reg_n[:5]):
            if self.secured[i] == '.':
                final += n + ']'
            else:
                final += f'{self.secured[i]}'
        return re.compile(final + '$')

    def feedback(self):
        result = input('write down the feedback from the game: \s "+" for the gray letter,"-" for the yellow letter and "LETTER" for the green letter')
        re_1 = self.regex_exp_one(result)
        self.filtered_words = list(filter(re_1.match, self.filtered_words))
        r_2 = self.regex_exp_two()
        self.filtered_words = list(filter(r_2.match, self.filtered_words))

    def type_word(self):
        self.guess = input('type your guess')

    def game_on_function(self):
        feed = input('Did you win?: Type "yes" or "no"')
        while feed not in ['Yes', 'yes', 'no', 'No']:
            print('Wrong input, try again')
            feed = input('Did you win?: Type "yes" or "no"')
        if feed == 'yes' or feed == 'Yes':
            return False
        elif feed == 'No' or feed == 'no':
            return True

In [58]:
game = BestGuess()

In [ ]:
game_on = True
game.type_word()
while len(game.filtered_words) != 1 and game_on :
    game.feedback()
    game_on = game.game_on_function()
    if not game_on:
        break
    game.guess = game.highest_entropy_word()[0]
    print(f'your next best word is: {game.guess}')

type your guess crane


# Find Best starting word
## Is 'TARES' by the way

In [31]:
test = BestGuess()
best = test.highest_entropy_word()
print('The best word is:', best[0])